# Извлечение векторных представлений из моделей трансформеров

В этом туториале мы рассмотрим, как извлекать векторные представления (также известные как эмбеддинги) из предобученных моделей трансформеров, таких как BERT, RoBERTa и DistilBERT. Эти векторные представления могут быть использованы в качестве входных данных для других моделей машинного обучения или для анализа и визуализации текстовых данных.

In [ ]:
import numpy as np

import torch

from torch.utils.data import DataLoader, Subset

from transformers import BertTokenizer, BertModel, DataCollatorWithPadding

from datasets import load_dataset

from tqdm import tqdm

## Загрузка данных

Мы начнем с загрузки датасета IMDb, который содержит отзывы на фильмы. Мы будем использовать только обучающую выборку для этого туториала.

In [ ]:
dataset = load_dataset("imdb", split="train")

## Подготовка данных

Для демонстрации мы будем использовать только подмножество данных. Мы случайным образом выберем 200 примеров из нашего датасета. Затем мы определим устройство, на котором будут выполняться наши вычисления (GPU, если доступно, иначе CPU).

In [ ]:
np.random.seed(100)

idx = np.random.randint(len(dataset), size=200)

index_list = idx.tolist()



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print("Device:", device)

if device.type == 'cuda':

    print("GPU:", torch.cuda.get_device_name())

## Загрузка модели BERT

Мы начнем с загрузки предобученной модели BERT. Мы будем использовать базовую версию модели BERT, обученную на английском языке.

In [ ]:
model = BertModel.from_pretrained("bert-base-cased").to(device)

## Токенизация

Прежде чем мы сможем использовать наши текстовые данные с моделью BERT, нам нужно преобразовать их в формат, который модель может понять. Это процесс называется токенизацией. Мы будем использовать токенизатор, соответствующий нашей модели BERT.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def tokenization(example):

    return tokenizer.batch_encode_plus(example['text'], add_special_tokens=True, return_token_type_ids=False,

                                       truncation=True)

## Подготовка данных для модели

Теперь, когда у нас есть функция для токенизации наших данных, мы можем применить ее к нашему датасету. Затем мы установим формат нашего датасета на "torch", чтобы мы могли использовать его с нашей моделью PyTorch.

In [ ]:
subset = Subset(dataset, index_list)

subset = subset.map(tokenization, batched=True, batch_size=len(subset))

subset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

## Извлечение эмбеддингов

Теперь, когда наши данные готовы, мы можем применить нашу модель BERT к ним, чтобы получить эмбеддинги. Мы будем использовать DataLoader для батчевой обработки наших данных. Затем мы пройдемся по каждому батчу, применим модель и сохраним полученные эмбеддинги.

In [ ]:
dataloader = DataLoader(subset, batch_size=16, shuffle=False)

embeddings = []

model.eval()

with torch.no_grad():

    for batch in tqdm(dataloader):

        input_ids = batch['input_ids'].to(device)

        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask)

        embeddings.append(outputs.last_hidden_state[:, 0, :].detach().cpu().numpy())

embeddings = np.concatenate(embeddings)

## Результаты

Теперь у нас есть векторные представления для каждого примера в нашем датасете. Каждое представление имеет размерность 768, что соответствует размерности скрытого состояния модели BERT, которую мы использовали.

In [ ]:
print("Embeddings shape:", embeddings.shape)